In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-Coursera/laptop_pricing_dataset_mod2.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,Manufacturer,Category,GPU,OS,CPU_core,Screen_Size_inch,CPU_frequency,RAM_GB,Storage_GB_SSD,Weight_pounds,Price,Price-binned,Screen-Full_HD,Screen-IPS_panel
0,0,0,Acer,4,2,1,5,14.0,0.551724,8,256,3.52800,978,Low,0,1
1,1,1,Dell,3,1,1,3,15.6,0.689655,4,256,4.85100,634,Low,1,0
2,2,2,Dell,3,1,1,7,15.6,0.931034,8,256,4.85100,946,Low,1,0
3,3,3,Dell,4,2,1,5,13.3,0.551724,8,128,2.69010,1244,Low,0,1
4,4,4,HP,4,2,1,7,15.6,0.620690,8,256,4.21155,837,Low,1,0


In [3]:
df.drop(labels=['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)

### Task 1: Using Cross validation to improve the model

In [4]:
x_data = df.drop(labels=['Price'], axis=1)
y_data = df['Price']

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=1)
print('x_train.shape: ', x_train.shape)
print('y_train.shape: ', y_train.shape)
print('x_test.shape: ', x_test.shape)
print('y_test.shape: ', y_test.shape)

x_train.shape:  (214, 13)
y_train.shape:  (214,)
x_test.shape:  (24, 13)
y_test.shape:  (24,)


In [6]:
from sklearn.linear_model import LinearRegression
slr = LinearRegression()
slr.fit(x_train[['CPU_frequency']], y_train)
r2_score_train = slr.score(x_train[['CPU_frequency']], y_train)
r2_score_test = slr.score(x_test[['CPU_frequency']], y_test)

In [9]:
from sklearn.model_selection import cross_val_score
slr = LinearRegression()
slr.fit(x_train[['CPU_frequency']], y_train)
cross_val_scores = cross_val_score(slr, x_train[['CPU_frequency']], y_train)
print(f'Folds:\n\tMean: {np.mean(cross_val_scores)}\n\tStddev: {np.std(cross_val_scores)}')

Folds:
	Mean: 0.10316999589149192
	Stddev: 0.09162531299331605
